In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data= pd.read_csv('loan_sanction_train.csv')

In [3]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
data.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
data.dropna(axis=0,inplace=True)

In [6]:
data=data.drop(['Loan_ID'],axis=1)

In [7]:
# clean data

In [8]:
from sklearn.preprocessing import LabelEncoder


le= LabelEncoder()
category_features=['Gender','Married','Education','Self_Employed','Loan_Status','Property_Area']
for column in data[category_features]:
    data[column]= le.fit_transform(data[column])
    
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1
5,1,1,2,0,1,5417,4196.0,267.0,360.0,1.0,2,1


In [9]:
data['Dependents'] = data['Dependents'].replace('3+', 3).astype('int')

In [10]:
x=data.drop(['Loan_Status'],axis=1)
y=data['Loan_Status']

print(x.shape)
print(y.shape)

(480, 11)
(480,)


In [21]:
# Normailizing the dataset to values between 0 and 1
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler= MinMaxScaler()
#scaler=StandardScaler()
x=scaler.fit_transform(x)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [23]:
x_train,x_test,y_train,y_test= train_test_split(x,y, test_size =.20,random_state=2)

# Building the ML Model and playing with the Grid Search CV for hyperparameter tuning for KNN

In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
# training the model without the use of gridsearch
model_knn = KNeighborsClassifier()
model_knn.fit(x_train,y_train)
# score of KNN Model
print(model_knn.score(x_test,y_test))

0.7708333333333334


### Training model with Gridsearch CV

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
# hyperparameter tuning
grid_params = { 'n_neighbors' : [5,6,7,8,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=10, n_jobs = -1)

In [28]:
gs.fit(x_train,y_train)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


GridSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['minkowski', 'euclidean', 'manhattan'],
                         'n_neighbors': [5, 6, 7, 8, 9, 11, 13, 15],
                         'weights': ['uniform', 'distance']},
             verbose=1)

In [29]:
gs.best_params_

{'metric': 'minkowski', 'n_neighbors': 5, 'weights': 'uniform'}

In [30]:
gs.score(x_test,y_test)

0.7708333333333334

### from my observation, I noticed that the performance of the model(with gridsearch and without) both have the same score.